# HybridSolver — Math & Code Explained from Scratch

**Audience:** You know how to multiply matrices and solve `Ax = b`. You do NOT need to know physics.

---

## What problem does this solver solve?

Imagine you have **objects** ("bodies") that can move and rotate in 3D space. Some objects are connected by **rules** ("constraints") — for example, *"these two points must always stay 2 meters apart"* (a rigid rod).

The solver answers: **Given the forces on each object, what accelerations should each object have so that all the rules stay satisfied?**

### The big equation

$$M \cdot a = F_{\text{applied}} + J^T \cdot \lambda$$
$$J \cdot a = \text{rhs}$$

| Symbol | What it is | Size |
|--------|-----------|------|
| $M$ | Mass matrix — how hard each object is to accelerate | (6N × 6N) |
| $a$ | Accelerations we want to find | (6N,) |
| $F$ | Applied forces (gravity, etc.) | (6N,) |
| $J$ | Constraint Jacobian — how constraints depend on velocities | (m × 6N) |
| $\lambda$ | Lagrange multipliers — unknown constraint force magnitudes | (m,) |
| rhs | Target for constraint accelerations (with stabilization) | (m,) |
| N | Number of bodies | scalar |
| m | Number of constraint equations | scalar |

**Why 6 per body?** Each body has 6 degrees of freedom: 3 for movement (x, y, z) and 3 for rotation (around x, y, z axes).

In [ ]:
import numpy as np
np.set_printoptions(precision=4, suppress=True, linewidth=120)

from aerislab.dynamics.body import RigidBody6DOF
from aerislab.dynamics.constraints import DistanceConstraint
from aerislab.core.solver import assemble_system, solve_kkt, HybridSolver

## Our example setup

Two objects connected by a vertical rigid rod (2 m long):
- **Anchor** (very heavy, 1000 kg) at position (0, 0, 2) — like a ceiling hook
- **Payload** (5 kg) at position (0, 0, 0) — hanging below

Gravity pulls the payload down. The rod must hold it.

```
  ████ Anchor (1000 kg)    z = 2 m
    │
    │  rod (L = 2 m)
    │
    ● Payload (5 kg)       z = 0 m
    ↓ gravity
```

In [ ]:
q_id = np.array([0., 0., 0., 1.])  # identity orientation (no rotation)

anchor = RigidBody6DOF(
    name="Anchor", mass=1000.0,
    inertia_tensor_body=np.diag([100., 100., 100.]),
    position=np.array([0., 0., 2.]),
    orientation=q_id,
)
payload = RigidBody6DOF(
    name="Payload", mass=5.0,
    inertia_tensor_body=np.diag([0.5, 0.5, 0.5]),
    position=np.array([0., 0., 0.]),
    orientation=q_id,
)
bodies = [anchor, payload]

# Apply gravity: F = mass × g
g = np.array([0., 0., -9.81])
for b in bodies:
    b.clear_forces()
    b.apply_force(b.mass * g)

# Distance constraint: keep them 2 m apart
constraint = DistanceConstraint(
    world_bodies=bodies, body_i=0, body_j=1,
    attach_i_local=np.zeros(3), attach_j_local=np.zeros(3),
    length=2.0,
)
constraints = [constraint]

print(f"Anchor:  mass={anchor.mass} kg,  pos={anchor.p}")
print(f"Payload: mass={payload.mass} kg,  pos={payload.p}")
print(f"Gravity force on payload: {payload.f} N")
print(f"Distance between them: {np.linalg.norm(anchor.p - payload.p):.1f} m")

---
# Part 1: `assemble_system` — Building the matrices

This function builds all the matrices needed for the KKT system. Let's go through it line by line.

## 1.1 The Inverse Mass Matrix `M⁻¹`

**Intuition:** The mass matrix says "how hard is it to push this object?" A heavier object is harder to accelerate. Newton's law: $a = F / m$, or in matrix form: $a = M^{-1} \cdot F$.

Each body gets a 6×6 block (3 linear DOFs + 3 rotational DOFs):

$$W_i = \begin{bmatrix} \frac{1}{m} I_3 & 0 \\ 0 & I_{\text{rot}}^{-1} \end{bmatrix}$$

- Top-left 3×3: `(1/mass) × identity` — how easily the body moves linearly
- Bottom-right 3×3: inverse of the "inertia tensor" — how easily the body rotates

The global matrix is **block-diagonal** (bodies don't share mass):

$$M^{-1} = \begin{bmatrix} W_0 & 0 \\ 0 & W_1 \end{bmatrix}$$

In [ ]:
nb = len(bodies)   # 2 bodies
nv = 6 * nb        # 12 DOFs total

Minv = np.zeros((nv, nv))
F_vec = np.zeros(nv)
v_vec = np.zeros(nv)

for i, b in enumerate(bodies):
    Wi = b.inv_mass_matrix_world()
    Minv[6*i:6*i+6, 6*i:6*i+6] = Wi
    print(f"--- {b.name} (body {i}, DOFs {6*i}..{6*i+5}) ---")
    print(f"  1/mass = {b.inv_mass:.6f}")
    print(f"  Inverse mass block W_{i}:\n{Wi}\n")

print("Full M⁻¹ (12×12):")
print(Minv)

## 1.2 The Generalized Force Vector `F`

For each body, we stack `[force_x, force_y, force_z, torque_x, torque_y, torque_z]` into one big vector.

There's also a correction called **gyroscopic torque**: $\tau_{\text{gyro}} = -\omega \times (I \cdot \omega)$

**What is this?** When an object spins, its rotation axis can "wobble" due to the way angular momentum works. This formula accounts for that. When the object is not spinning ($\omega = 0$), this term is zero.

In [ ]:
for i, b in enumerate(bodies):
    I_world = b.inertia_world()
    tau_gyro = -np.cross(b.w, I_world @ b.w)
    
    F_gen = b.generalized_force()   # [force; torque]
    F_gen[3:6] += tau_gyro          # add gyroscopic correction to torque part
    
    F_vec[6*i:6*i+6] = F_gen
    v_vec[6*i:6*i+3] = b.v          # linear velocity
    v_vec[6*i+3:6*i+6] = b.w        # angular velocity
    
    print(f"--- {b.name} ---")
    print(f"  ω (spin rate) = {b.w}  →  gyroscopic torque = {tau_gyro}")
    print(f"  Generalized force = {F_gen}")
    print(f"    ↳ first 3: force [N],  last 3: torque [N·m]\n")

print(f"Global F vector (12,): {F_vec}")
print(f"Global v vector (12,): {v_vec}")

## 1.3 The Constraint Jacobian `J`

**The key idea:** A constraint is a rule like $C(\text{positions}) = 0$. Its time derivative gives:

$$\dot{C} = J \cdot v$$

The Jacobian $J$ tells us: *"If I change each velocity by a tiny amount, how much does the constraint violation change?"*

For our distance constraint $C = \frac{1}{2}(\|d\|^2 - L^2)$, the Jacobian row is:

$$J = [d^T, \; (r_A \times d)^T, \; -d^T, \; -(r_B \times d)^T]$$

where $d = p_A - p_B$ is the vector between the bodies.

Each constraint produces a **local Jacobian** (columns for its 2 connected bodies), which gets **scattered** into the correct columns of the **global** Jacobian.

In [ ]:
m = sum(c.rows() for c in constraints)   # total constraint equations
J = np.zeros((m, nv))
rhs = np.zeros(m)

row = 0
for c in constraints:
    r = c.rows()
    bi, bj = c.index_map()
    Jloc = c.jacobian()  # (r × 12) matrix
    
    # Scatter into global J
    J[row:row+r, 6*bi:6*bi+6] = Jloc[:, 0:6]    # body i columns
    J[row:row+r, 6*bj:6*bj+6] = Jloc[:, 6:12]   # body j columns
    
    d = anchor.p - payload.p
    print(f"Direction vector d = p_anchor - p_payload = {d}")
    print(f"\nLocal Jacobian (1×12):")
    print(f"  Body {bi} linear  cols 0-2:  {Jloc[0,0:3]}   (= d)")
    print(f"  Body {bi} angular cols 3-5:  {Jloc[0,3:6]}   (= r_A × d)")
    print(f"  Body {bj} linear  cols 6-8:  {Jloc[0,6:9]}   (= -d)")
    print(f"  Body {bj} angular cols 9-11: {Jloc[0,9:12]}  (= -(r_B × d))")
    print(f"\nGlobal J ({m}×{nv}):")
    print(J)
    row += r

## 1.4  Baumgarte Stabilization (the constraint RHS)

**The problem:** Even if we enforce $\dot{C} = 0$ (constraint velocity = 0), numerical errors will slowly cause the constraint to drift. The bodies will gradually move apart from the required distance.

**The fix (Baumgarte, 1972):** Add two correction terms:

$$\text{rhs} = \underbrace{-(1 + \beta) \cdot J v}_{\text{velocity damping}} \;\underbrace{- \alpha \cdot C}_{\text{position spring}}$$

| Term | Role | Analogy |
|------|------|---------|
| $-(1+\beta) \cdot Jv$ | Kills constraint velocity | A **damper** that slows down any constraint drift |
| $-\alpha \cdot C$ | Pushes violation back to zero | A **spring** that pulls things back if they drifted |

- $\alpha$ = how aggressively to correct position errors (bigger = stronger spring)
- $\beta$ = how aggressively to correct velocity errors (bigger = stronger damper)

In [ ]:
alpha, beta = 5.0, 1.0

row = 0
for c in constraints:
    r = c.rows()
    bi, bj = c.index_map()
    Jloc = c.jacobian()
    
    v_loc = np.concatenate([v_vec[6*bi:6*bi+6], v_vec[6*bj:6*bj+6]])
    Jv = Jloc @ v_loc        # constraint velocity
    C = c.evaluate()         # constraint violation
    
    rhs[row:row+r] = -(1.0 + beta) * Jv - alpha * C
    
    print(f"Constraint velocity Jv = {Jv}   (how fast the constraint is being violated)")
    print(f"Constraint violation C  = {C}   (how far from satisfied)")
    print(f"")
    print(f"Baumgarte RHS = -(1+{beta})×{Jv[0]:.4f} - {alpha}×{C[0]:.4f} = {rhs[row:row+r]}")
    print(f"  Both are zero here — the constraint is perfectly satisfied and nothing is moving.")
    row += r

### ✅ Verify: our manual computation matches `assemble_system`

In [ ]:
Minv_check, J_check, F_check, rhs_check, v_check = assemble_system(bodies, constraints, alpha, beta)
assert np.allclose(Minv, Minv_check) and np.allclose(J, J_check)
assert np.allclose(F_vec, F_check) and np.allclose(rhs, rhs_check)
print("✓ All matrices match the assemble_system() output!")

---
# Part 2: `solve_kkt` — Finding the constrained accelerations

We have two unknowns: accelerations $a$ and constraint forces $\lambda$.  
We use the **Schur complement** method — a 4-step recipe to solve the system.

### Step 1: What would happen without any constraints?

In [ ]:
# Step 1: Unconstrained acceleration
a0 = Minv @ F_vec

print("If there were NO rod connecting them:")
print(f"  Anchor  linear accel: {a0[0:3]} m/s²  (barely moves — very heavy)")
print(f"  Payload linear accel: {a0[6:9]} m/s²  (free-falls at 9.81 m/s²)")

### Step 2: Effective constraint mass $A = J \cdot M^{-1} \cdot J^T$

This tells us: *"Along the constraint direction, how easy is it to accelerate the system?"*  
It combines the masses of both connected bodies, projected onto the constraint direction.

In [ ]:
A = J @ (Minv @ J.T)
print(f"Effective mass A = J·M⁻¹·Jᵀ = {A}")
print(f"  This is a 1×1 scalar because we have 1 constraint equation.")

### Step 3: Solve for $\lambda$ (constraint force magnitude)

$$A \cdot \lambda = \text{rhs} - J \cdot a_0$$

This is a standard linear system `Ax = b`.

In [ ]:
b_vec = rhs - J @ a0
lam = np.linalg.solve(A, b_vec)

print(f"b = rhs - J·a₀ = {b_vec}")
print(f"λ = A⁻¹·b = {lam}")
print(f"\nWhat does λ mean?")
print(f"  It's the tension in the rod needed to prevent the payload from falling.")

### Step 4: Final acceleration $a = a_0 + M^{-1} \cdot J^T \cdot \lambda$

In [ ]:
a = a0 + Minv @ (J.T @ lam)

print("Constrained accelerations:")
print(f"  Anchor  linear: {a[0:3]} m/s²")
print(f"  Payload linear: {a[6:9]} m/s²")
print(f"\nThe rod transfers weight: the payload barely falls,")
print(f"and the anchor is pulled down slightly by the payload's weight.")

# Verify
a_check, lam_check = solve_kkt(Minv, J, F_vec, rhs)
assert np.allclose(a, a_check) and np.allclose(lam, lam_check)
print("\n✓ Matches solve_kkt() output!")

---
# Part 3: `HybridSolver.step()` — Time integration

Now that we can compute accelerations at any instant, we need to **advance time**.

The `HybridSolver` uses **semi-implicit Euler** integration with a fixed time step `dt`:

| Step | Formula | What it does |
|------|---------|--------------|
| 1 | $v_{n+1} = v_n + a \cdot dt$ | Update velocity using computed acceleration |
| 2 | $p_{n+1} = p_n + v_{n+1} \cdot dt$ | Update position using the **new** velocity |
| 3 | $q_{n+1} = \text{normalize}(q_n + \dot{q} \cdot dt)$ | Update rotation (quaternion) |

**Why "semi-implicit"?** Step 2 uses the *updated* velocity $v_{n+1}$ (not the old $v_n$). This small trick gives much better energy conservation — the simulation doesn't "explode" over time.

### Let's simulate 1 second (100 steps × 0.01s)

In [ ]:
# Fresh setup
anchor2 = RigidBody6DOF("Anchor", 1000.0, np.diag([100.,100.,100.]),
                         np.array([0.,0.,2.]), q_id)
payload2 = RigidBody6DOF("Payload", 5.0, np.diag([0.5,0.5,0.5]),
                          np.array([0.,0.,0.]), q_id)
bodies2 = [anchor2, payload2]
constr2 = DistanceConstraint(bodies2, 0, 1, np.zeros(3), np.zeros(3), 2.0)

solver = HybridSolver(alpha=5.0, beta=1.0)
dt = 0.01

# Record trajectory
history_z_anchor = []
history_z_payload = []
history_dist = []

for step in range(100):
    # Apply gravity each step
    for b in bodies2:
        b.clear_forces()
        b.apply_force(b.mass * g)
    
    solver.step(bodies2, [constr2], dt)
    
    history_z_anchor.append(anchor2.p[2])
    history_z_payload.append(payload2.p[2])
    history_dist.append(np.linalg.norm(anchor2.p - payload2.p))

print(f"After 1 second (100 steps):")
print(f"  Anchor  position: {anchor2.p}")
print(f"  Payload position: {payload2.p}")
print(f"  Distance between them: {np.linalg.norm(anchor2.p - payload2.p):.6f} m")
print(f"  Required distance:     2.000000 m")
print(f"  → Constraint is maintained! ✓")

In [ ]:
import matplotlib.pyplot as plt

t = np.arange(1, 101) * dt
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(t, history_z_anchor, label='Anchor z')
axes[0].plot(t, history_z_payload, label='Payload z')
axes[0].set_xlabel('Time [s]')
axes[0].set_ylabel('Height z [m]')
axes[0].set_title('Heights over time')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(t, history_dist, 'r-')
axes[1].axhline(2.0, color='k', linestyle='--', alpha=0.5, label='Required distance')
axes[1].set_xlabel('Time [s]')
axes[1].set_ylabel('Distance [m]')
axes[1].set_title('Constraint satisfaction')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
# Summary: The full pipeline

```
Each time step:
│
├─ 1. assemble_system()
│    ├─ Build M⁻¹ (how easy to accelerate each body)
│    ├─ Build F   (all forces: gravity + gyroscopic)
│    ├─ Build J   (how constraints depend on velocities)
│    └─ Build rhs (Baumgarte spring + damper correction)
│
├─ 2. solve_kkt()
│    ├─ a₀ = M⁻¹·F               (free-fall acceleration)
│    ├─ A  = J·M⁻¹·Jᵀ             (effective constraint mass)
│    ├─ λ  = A⁻¹·(rhs - J·a₀)    (how much force the constraints need)
│    └─ a  = a₀ + M⁻¹·Jᵀ·λ       (corrected acceleration)
│
└─ 3. integrate (semi-implicit Euler)
     ├─ v_new = v + a·dt
     ├─ p_new = p + v_new·dt
     └─ q_new = normalize(q + q̇·dt)
```